In [2]:
import os
import pickle

import librosa
import librosa.display
import IPython.display
import numpy as np
import scipy.signal

import matplotlib.style as ms
import matplotlib.pyplot as plt
ms.use('seaborn-muted')
%matplotlib inline

from collections import Counter, defaultdict
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

In [3]:
def get_Chromagram_from_spectragram(S, gain, sr=22050, to_plot=True, method='CLP'):

    C = librosa.feature.chroma_stft(S=np.log(1.0 + gain * np.abs(S)), sr=sr)

    plt_title = 'Chromagram('+method.upper()+') - γ='+str(gain)
        
    if to_plot:
        # Make a new figure
        plt.figure(figsize=(14,4))

        # Display the chromagram: the energy in each chromatic pitch class as a function of time
        # To make sure that the colors span the full range of chroma values, set vmin and vmax
        librosa.display.specshow(C, sr=sr, x_axis='time', y_axis='chroma', vmin=0)

        plt.title(plt_title)
        plt.colorbar()
        plt.tight_layout()

    return C

In [4]:
def get_key(chroma, key_template, method='sum', medfilter=False, blue_tone=None):
    if method == 'dot':
        if blue_tone:
            chroma[6]=np.zeros(chroma.shape[1])
        chroma = chroma/np.tile(np.sum(np.abs(chroma)**2, axis=0)**(1./2), \
                                (chroma.shape[0], 1)) # normalized chroma by the square sum of each time point
        dot_result = np.dot(key_template, chroma) # dot product with chroma and template

        tonic = dot_result.argmax(axis=0) # tonic in each time point
        
        if medfilter:
            tonic = scipy.signal.medfilt(tonic, kernel_size = 21) # median filter as post-processing
        
        key_count_dict = Counter(tonic)
        tonic = int(key_count_dict.most_common()[0][0])
        
        return tonic
    
    elif method == 'sum':
        summed_chroma = np.sum(chroma, axis=1) # sum up all the chroma
        if blue_tone: summed_chroma[6]=0
        tonic = summed_chroma.argmax()
        maj_cc = np.dot(key_template[tonic], summed_chroma)
        min_cc = np.dot(key_template[tonic+12], summed_chroma)
        
        if maj_cc > min_cc: # Major Key
            return tonic 
        else: # Minor Key
            return tonic+12
        
def note_converter(tonic):
    return (tonic+3)%12 if tonic/12 < 1 else ((tonic+3)%12)+12

## Q4: Krumhansl-Schmuckler's algorithm

In [7]:
# Generate major key templates
# C maj
KS_major_template = np.array([[6.35,2.23,3.48,2.33,4.38,4.09,2.52,5.19,2.39,3.66,2.29,2.88]])
KS_major_template = KS_major_template/np.sqrt(np.dot(KS_major_template, KS_major_template.T))
# Generate monor key templates
# C min
KS_minor_template = np.array([[6.33,2.68,3.52,5.38,2.60,3.53,2.54,4.75,3.98,2.69,3.34,3.17]]) 
KS_minor_template = KS_minor_template/np.sqrt(np.dot(KS_minor_template, KS_minor_template.T))

KS_key_template = KS_major_template
for i in range(11):
    KS_key_template = np.append(KS_key_template, np.roll(KS_major_template, i+1), axis=0)    
for i in range(12):
    KS_key_template = np.append(KS_key_template, np.roll(KS_minor_template, i), axis=0)
    
KS_key_template.shape

(24, 12)

In [49]:
def key_detection_pipeline_pre(feature_dict, 
                               gammas=[100],
                               genres = genres,
                               weight_eval = False,
                               key_template = KS_key_template,
                               y_source='source',
                               match_method='sum',
                               medfilter=False, 
                               plot_chroma=False, 
                               verbose = False,
                               blue_tone = False,
                               out_file = None
                              ):
       
    guesses = defaultdict(lambda:defaultdict(lambda:[])) # dictionary to save the precict result for each gamma
    labels  = defaultdict(lambda:defaultdict(lambda:[])) # dictionary to save the true label for each gamma
    
    if out_file:
        out = open(out_file,'w')
    for genre in feature_dict:
        print(genre)
        if blue_tone and genre=="blues":
            is_blue = True
        else:
            is_blue = False
        for gamma in gammas:
            if out_file:
                out.write("\nGamma: " + str(gamma) + "\n")
            for i, music_spectragram in enumerate(feature_dict[genre]['features']):
                chroma = get_Chromagram_from_spectragram(music_spectragram, gamma, sr=22050, to_plot=plot_chroma)
                if is_blue:
                    chroma[6] = np.zeros(chroma.shape[1])
                key = get_key(chroma, key_template, method=match_method, medfilter=medfilter)
                key = note_converter(key) # fit to label, because label is from A, ours are from C
                
                guesses[gamma][genre].append(key)
                
                if verbose:
                    print("{:8s}({:14s})\tGUESS:{:3d}, LABEL:{:3d}".format(genre.upper(), feature_dict[genre]['filenames'][i], key, feature_dict[genre]['labels'][i]))
                if out_file:
                    out.write("{:8s}\t{:14s}\tGUESS:{:3d}, LABEL:{:3d}\n".format(genre.upper(), feature_dict[genre]['filenames'][i], key, feature_dict[genre]['labels'][i]))
            labels[gamma][genre] = feature_dict[genre]['labels']
            
    # Evaluation
    for gamma in gammas:
        print("Gamma: " + str(gamma))
        evaluation(labels[gamma], guesses[gamma], genres, weighted=weight_eval)

perfect_fifth_dict = {3:10, 10:5, 5:0, 0:7, 7:2, 2:9, 9:4, 4:11, 11:6, 6:1, 1:8, 8:3, 
                      12:19,19:14, 14:21, 21:16, 16:23, 23:18, 18:13, 13:20, 20:15, 15:22, 22:17, 17:12}
relative_dict = {3:12, 12:3, 10:19, 19:10, 5:14, 14:5, 0:21, 21:0, 7:16, 16:7, 2:23, 23:2,  
                 9:18, 18:9, 4:13, 13:4, 11:20, 20:11, 6:15, 15:6, 1:22, 22:1, 8:17, 17:8}
    
def evaluation(y_true_dict, 
               y_predict_dict, 
               genres = genres, 
               weighted = False, 
               to_classification_report=False, 
               target_names = ["A","A#","B","C","C#","D","D#","E","F","F#","G","G#","a","a#","b","c","c#","d","d#","e","f","f#","g","g#"], 
               perfect_fifth_dict = perfect_fifth_dict, 
               relative_dict = relative_dict):
    # classification report
    if to_classification_report:
        for genre in genres:
            print(genre)
            print(classification_report(y_true_dict[genre], y_predict_dict[genre], target_names=target_names))
            print()
    class_acc = []
    if weighted:
        print("Weighted accuracy")
        for genre in genres:
            correct = 0.0
            for i, label in enumerate(y_true_dict[genre]):
                if y_predict_dict[genre][i] == label: # same
                    correct += 1
                if perfect_fifth_dict[y_predict_dict[genre][i]] == label: # perfect fifth error
                    correct += 0.5
                if relative_dict[y_predict_dict[genre][i]] == label: # Relative major/minor error
                    correct += 0.3
                if (y_predict_dict[genre][i]%12) == (label%12): # parallel major/minor
                    correct += 0.2
            class_acc.append(correct/len(y_predict_dict[genre]))
            print("{:8s}\t{:.2f}%".format(genre.upper(), correct/len(y_predict_dict[genre])*100))
    else:
        print("Average accuracy")
        for genre in genres:
            correct = 0.0
            for i, label in enumerate(y_true_dict[genre]):
                if label == y_predict_dict[genre][i]:
                    correct += 1
            try:
                class_acc.append(correct/len(y_predict_dict[genre]))
            except:class_acc.append(0.0)
            print("{:8s}\t{:.2f}%".format(genre.upper(), correct/len(y_predict_dict[genre])*100))
    print("{:8s}\t{:.2f}%".format("ALL AVG", sum(class_acc)/len(class_acc)*100))

## Load Pre-compute Features

In [5]:
dataset_dir = "../datasets/genres"
genres = ["pop", "blues", "metal", "hiphop", "rock"]

# Read features from pickle
with open("../datasets/spectrum_features.pkl","rb") as f:
    feature_dict = pickle.load(f)
    
print("Features Loads")

Features Loads


## Experiments

In [12]:
%%time
key_detection_pipeline_pre(feature_dict, 
                           gammas = [1,10,100,1000],
                           key_template = KS_key_template,
                           out_file="Q4_different_gamma_sum_method.txt")
"""
Sum Method

Gamma: 1
Average accuracy
POP     	17.39%
BLUES   	28.57%
METAL   	16.30%
HIPHOP  	16.46%
ROCK    	16.67%
ALL AVG 	19.08%
Gamma: 10
Average accuracy
POP     	14.13%
BLUES   	27.55%
METAL   	11.96%
HIPHOP  	12.66%
ROCK    	11.11%
ALL AVG 	15.48%
Gamma: 100
Average accuracy
POP     	7.61%
BLUES   	22.45%
METAL   	9.78%
HIPHOP  	12.66%
ROCK    	7.78%
ALL AVG 	12.06%
Gamma: 1000
Average accuracy
POP     	7.61%
BLUES   	15.31%
METAL   	7.61%
HIPHOP  	7.59%
ROCK    	6.67%
ALL AVG 	8.96%
"""

Gamma: 1
Average accuracy
POP     	17.39%
BLUES   	28.57%
METAL   	16.30%
HIPHOP  	16.46%
ROCK    	16.67%
ALL AVG 	19.08%
Gamma: 10
Average accuracy
POP     	14.13%
BLUES   	27.55%
METAL   	11.96%
HIPHOP  	12.66%
ROCK    	11.11%
ALL AVG 	15.48%
Gamma: 100
Average accuracy
POP     	7.61%
BLUES   	22.45%
METAL   	9.78%
HIPHOP  	12.66%
ROCK    	7.78%
ALL AVG 	12.06%
Gamma: 1000
Average accuracy
POP     	7.61%
BLUES   	15.31%
METAL   	7.61%
HIPHOP  	7.59%
ROCK    	6.67%
ALL AVG 	8.96%
CPU times: user 37min 8s, sys: 2min 2s, total: 39min 11s
Wall time: 4min 55s


In [11]:
%%time
key_detection_pipeline_pre(feature_dict,
                           gammas = [1,10,100,1000], 
                           key_template = KS_key_template,
                           weight_eval = True,
                           out_file="Q4_different_gamma_sum_weight.txt")
"""
Sum Method with weight evaluation

Gamma: 1
Weighted accuracy
POP     	25.65%
BLUES   	37.76%
METAL   	24.13%
HIPHOP  	24.68%
ROCK    	25.00%
ALL AVG 	27.44%
Gamma: 10
Weighted accuracy
POP     	21.52%
BLUES   	36.53%
METAL   	21.09%
HIPHOP  	19.49%
ROCK    	18.56%
ALL AVG 	23.44%
Gamma: 100
Weighted accuracy
POP     	13.59%
BLUES   	30.71%
METAL   	14.67%
HIPHOP  	19.49%
ROCK    	14.22%
ALL AVG 	18.54%
Gamma: 1000
Weighted accuracy
POP     	13.37%
BLUES   	24.18%
METAL   	12.07%
HIPHOP  	13.80%
ROCK    	12.44%
ALL AVG 	15.17%
"""

Gamma: 1
Weighted accuracy
POP     	25.65%
BLUES   	37.76%
METAL   	24.13%
HIPHOP  	24.68%
ROCK    	25.00%
ALL AVG 	27.44%
Gamma: 10
Weighted accuracy
POP     	21.52%
BLUES   	36.53%
METAL   	21.09%
HIPHOP  	19.49%
ROCK    	18.56%
ALL AVG 	23.44%
Gamma: 100
Weighted accuracy
POP     	13.59%
BLUES   	30.71%
METAL   	14.67%
HIPHOP  	19.49%
ROCK    	14.22%
ALL AVG 	18.54%
Gamma: 1000
Weighted accuracy
POP     	13.37%
BLUES   	24.18%
METAL   	12.07%
HIPHOP  	13.80%
ROCK    	12.44%
ALL AVG 	15.17%
CPU times: user 37min 9s, sys: 1min 59s, total: 39min 8s
Wall time: 4min 54s


In [9]:
%%time
key_detection_pipeline_pre(feature_dict, 
                           gammas = [1,10,100,1000],
                           key_template = KS_key_template,
                           match_method='dot',
                           out_file="Q4_different_gamma_dot_method.txt")
"""
Dot Method

Gamma: 1
Average accuracy
POP     	23.91%
BLUES   	19.39%
METAL   	21.74%
HIPHOP  	16.46%
ROCK    	17.78%
ALL AVG 	19.85%
Gamma: 10
Average accuracy
POP     	21.74%
BLUES   	8.16%
METAL   	25.00%
HIPHOP  	16.46%
ROCK    	18.89%
ALL AVG 	18.05%
Gamma: 100
Average accuracy
POP     	16.30%
BLUES   	17.35%
METAL   	13.04%
HIPHOP  	12.66%
ROCK    	14.44%
ALL AVG 	14.76%
Gamma: 1000
Average accuracy
POP     	16.30%
BLUES   	18.37%
METAL   	7.61%
HIPHOP  	11.39%
ROCK    	10.00%
ALL AVG 	12.73%
"""

Gamma: 1
Average accuracy
POP     	23.91%
BLUES   	19.39%
METAL   	21.74%
HIPHOP  	16.46%
ROCK    	17.78%
ALL AVG 	19.85%
Gamma: 10
Average accuracy
POP     	21.74%
BLUES   	8.16%
METAL   	25.00%
HIPHOP  	16.46%
ROCK    	18.89%
ALL AVG 	18.05%
Gamma: 100
Average accuracy
POP     	16.30%
BLUES   	17.35%
METAL   	13.04%
HIPHOP  	12.66%
ROCK    	14.44%
ALL AVG 	14.76%
Gamma: 1000
Average accuracy
POP     	16.30%
BLUES   	18.37%
METAL   	7.61%
HIPHOP  	11.39%
ROCK    	10.00%
ALL AVG 	12.73%
CPU times: user 37min 8s, sys: 1min 57s, total: 39min 6s
Wall time: 4min 54s


In [25]:
%%time
key_detection_pipeline_pre(feature_dict, 
                           gammas = [1,10,100,1000],
                           key_template = KS_key_template,
                           match_method='dot',
                           weight_eval = True,
                           out_file="Q4_different_gamma_dot_weight.txt")

"""
Dot Method with weight evaluation

Gamma: 1
Weighted accuracy
POP     	35.65%
BLUES   	25.31%
METAL   	33.37%
HIPHOP  	23.80%
ROCK    	26.67%
ALL AVG 	28.96%
Gamma: 10
Weighted accuracy
POP     	34.89%
BLUES   	11.63%
METAL   	38.04%
HIPHOP  	21.90%
ROCK    	28.89%
ALL AVG 	27.07%
Gamma: 100
Weighted accuracy
POP     	27.83%
BLUES   	23.67%
METAL   	22.17%
HIPHOP  	17.72%
ROCK    	23.33%
ALL AVG 	22.95%
Gamma: 1000
Weighted accuracy
POP     	24.13%
BLUES   	25.31%
METAL   	13.48%
HIPHOP  	16.33%
ROCK    	17.56%
ALL AVG 	19.36%
"""

Gamma: 1
Weighted accuracy
POP     	35.65%
BLUES   	25.31%
METAL   	33.37%
HIPHOP  	23.80%
ROCK    	26.67%
ALL AVG 	28.96%
Gamma: 10
Weighted accuracy
POP     	34.89%
BLUES   	11.63%
METAL   	38.04%
HIPHOP  	21.90%
ROCK    	28.89%
ALL AVG 	27.07%
Gamma: 100
Weighted accuracy
POP     	27.83%
BLUES   	23.67%
METAL   	22.17%
HIPHOP  	17.72%
ROCK    	23.33%
ALL AVG 	22.95%
Gamma: 1000
Weighted accuracy
POP     	24.13%
BLUES   	25.31%
METAL   	13.48%
HIPHOP  	16.33%
ROCK    	17.56%
ALL AVG 	19.36%
CPU times: user 39min 55s, sys: 2min 28s, total: 42min 23s
Wall time: 5min 19s


## Q6 - Improve

### Temperley template

In [13]:
# Generate major key templates
# C maj
KP_major_template = np.array([[5,2,3.5,2,4.5,4.0,2.0,4.5,2.0,3.5,1.5,4.0]])
KP_major_template = KP_major_template/np.sqrt(np.dot(KP_major_template, KP_major_template.T))
# Generate monor key templates
# C min
KP_minor_template = np.array([[5.0,2.0,3.5,4.5,2.0,4.0,2.0,4.5,3.5,2.0,1.5,4.0]]) 
KP_minor_template = KP_minor_template/np.sqrt(np.dot(KP_minor_template, KP_minor_template.T))

KP_key_template = KP_major_template
for i in range(11):
    KP_key_template = np.append(KP_key_template, np.roll(KP_major_template, i+1), axis=0)    
for i in range(12):
    KP_key_template = np.append(KP_key_template, np.roll(KP_minor_template, i), axis=0)
    
KP_key_template.shape

(24, 12)

In [ ]:
# %%time
# key_detection_pipeline_pre(feature_dict, 
#                            gammas = [1,10,100,1000],
#                            key_template = KP_key_template,
#                            match_method='dot',
#                            weight_eval = True,
#                            out_file="Q5_Temperley_template.txt")

"""
Gamma: 1
Weighted accuracy
POP     	37.28%
BLUES   	14.59%
METAL   	28.48%
HIPHOP  	19.87%
ROCK    	38.44%
ALL AVG 	27.73%
Gamma: 10
Weighted accuracy
POP     	26.74%
BLUES   	12.86%
METAL   	27.93%
HIPHOP  	13.92%
ROCK    	43.11%
ALL AVG 	24.91%
Gamma: 100
Weighted accuracy
POP     	16.96%
BLUES   	13.57%
METAL   	27.72%
HIPHOP  	12.78%
ROCK    	26.56%
ALL AVG 	19.52%
Gamma: 1000
Weighted accuracy
POP     	14.89%
BLUES   	17.24%
METAL   	13.59%
HIPHOP  	11.27%
ROCK    	21.00%
ALL AVG 	15.60%
"""

In [14]:
%%time
key_detection_pipeline_pre(feature_dict, 
                           gammas = [1,10,100,1000],
                           key_template = KP_key_template,
                           match_method='sum',
                           weight_eval = True,
                           out_file="Q5_Temperley_template_sum.txt")

"""
Gamma: 1
Weighted accuracy
POP     	35.33%
BLUES   	12.04%
METAL   	19.89%
HIPHOP  	21.27%
ROCK    	28.78%
ALL AVG 	23.46%
Gamma: 10
Weighted accuracy
POP     	28.37%
BLUES   	11.63%
METAL   	20.33%
HIPHOP  	18.35%
ROCK    	24.11%
ALL AVG 	20.56%
Gamma: 100
Weighted accuracy
POP     	18.80%
BLUES   	13.47%
METAL   	17.07%
HIPHOP  	17.34%
ROCK    	15.33%
ALL AVG 	16.40%
Gamma: 1000
Weighted accuracy
POP     	19.35%
BLUES   	18.88%
METAL   	10.43%
HIPHOP  	14.56%
ROCK    	13.67%
ALL AVG 	15.38%
"""

Gamma: 1
Weighted accuracy
POP     	35.33%
BLUES   	12.04%
METAL   	19.89%
HIPHOP  	21.27%
ROCK    	28.78%
ALL AVG 	23.46%
Gamma: 10
Weighted accuracy
POP     	28.37%
BLUES   	11.63%
METAL   	20.33%
HIPHOP  	18.35%
ROCK    	24.11%
ALL AVG 	20.56%
Gamma: 100
Weighted accuracy
POP     	18.80%
BLUES   	13.47%
METAL   	17.07%
HIPHOP  	17.34%
ROCK    	15.33%
ALL AVG 	16.40%
Gamma: 1000
Weighted accuracy
POP     	19.35%
BLUES   	18.88%
METAL   	10.43%
HIPHOP  	14.56%
ROCK    	13.67%
ALL AVG 	15.38%
CPU times: user 38min 8s, sys: 2min, total: 40min 9s
Wall time: 5min 2s


### Cherry Picking method

In [43]:
def key_detection_pipeline_pre_cherry(feature_dict, 
                                       gammas=[100],
                                       genres = genres,
                                       weight_eval = False,
                                       key_template = KS_key_template,
                                       tem_template = KP_key_template,
                                       y_source='source',
                                       match_method='sum',
                                       medfilter=False, 
                                       plot_chroma=False, 
                                       verbose = False,
                                       blue_tone = False,
                                       out_file = None
                                      ):
       
    guesses = defaultdict(lambda:defaultdict(lambda:[])) # dictionary to save the precict result for each gamma
    labels  = defaultdict(lambda:defaultdict(lambda:[])) # dictionary to save the true label for each gamma
    
    if out_file:
        out = open(out_file,'w')
    for genre in feature_dict:
        if blue_tone and genre=="blues":
            match_method='dot'
            is_blue = True
        elif not blue_tone and genre=="blues":
            match_method='sum'
            is_blue = False
        else:
            match_method='dot'
            is_blue = False
        template = key_template if genre != 'rock' else tem_template
        for gamma in gammas:
            if out_file:
                out.write("\nGamma: " + str(gamma) + "\n")
            for i, music_spectragram in enumerate(feature_dict[genre]['features']):
                chroma = get_Chromagram_from_spectragram(music_spectragram, gamma, sr=22050, to_plot=plot_chroma)
                key = get_key(chroma, template, method=match_method, medfilter=medfilter, blue_tone=is_blue)
                key = note_converter(key) # fit to label, because label is from A, ours are from C
                
                guesses[gamma][genre].append(key)
                
                if verbose:
                    print("{:8s}({:14s})\tGUESS:{:3d}, LABEL:{:3d}".format(genre.upper(), feature_dict[genre]['filenames'][i], key, feature_dict[genre]['labels'][i]))
                if out_file:
                    out.write("{:8s}\t{:14s}\tGUESS:{:3d}, LABEL:{:3d}\n".format(genre.upper(), feature_dict[genre]['filenames'][i], key, feature_dict[genre]['labels'][i]))
            labels[gamma][genre] = feature_dict[genre]['labels']
            
    # Evaluation
    for gamma in gammas:
        print("Gamma: " + str(gamma))
        evaluation(labels[gamma], guesses[gamma], genres, weighted=weight_eval)

In [26]:
%%time
key_detection_pipeline_pre_cherry(feature_dict, 
                           gammas = [1,10,100,1000],
                           key_template = KS_key_template,
                           tem_template = KP_key_template,
                           match_method='dot',
                           weight_eval = True,
                           out_file="Q5_cherry_picking.txt")

"""
Gamma: 1
Weighted accuracy
POP     	35.65%
BLUES   	37.76%
METAL   	33.37%
HIPHOP  	23.80%
ROCK    	38.44%
ALL AVG 	33.80%
Gamma: 10
Weighted accuracy
POP     	34.89%
BLUES   	36.53%
METAL   	38.04%
HIPHOP  	21.90%
ROCK    	43.11%
ALL AVG 	34.90%
Gamma: 100
Weighted accuracy
POP     	27.83%
BLUES   	30.71%
METAL   	22.17%
HIPHOP  	17.72%
ROCK    	26.56%
ALL AVG 	25.00%
Gamma: 1000
Weighted accuracy
POP     	24.13%
BLUES   	24.18%
METAL   	13.48%
HIPHOP  	16.33%
ROCK    	21.00%
ALL AVG 	19.82%
"""

Gamma: 1
Weighted accuracy
POP     	35.65%
BLUES   	37.76%
METAL   	33.37%
HIPHOP  	23.80%
ROCK    	38.44%
ALL AVG 	33.80%
Gamma: 10
Weighted accuracy
POP     	34.89%
BLUES   	36.53%
METAL   	38.04%
HIPHOP  	21.90%
ROCK    	43.11%
ALL AVG 	34.90%
Gamma: 100
Weighted accuracy
POP     	27.83%
BLUES   	30.71%
METAL   	22.17%
HIPHOP  	17.72%
ROCK    	26.56%
ALL AVG 	25.00%
Gamma: 1000
Weighted accuracy
POP     	24.13%
BLUES   	24.18%
METAL   	13.48%
HIPHOP  	16.33%
ROCK    	21.00%
ALL AVG 	19.82%
CPU times: user 39min 49s, sys: 2min 30s, total: 42min 20s
Wall time: 5min 19s


## Blue tone remove

In [53]:
%%time
key_detection_pipeline_pre(feature_dict, 
                           gammas = [1,10,100,1000],
                           key_template = KS_key_template,
                           match_method='sum',
                           blue_tone=True,
                           weight_eval = True,
                           out_file="Q5_blue_tone_remove_sum_weight.txt")

pop
blues
metal
hiphop
rock
Gamma: 1
Weighted accuracy
POP     	25.65%
BLUES   	34.90%
METAL   	24.13%
HIPHOP  	24.68%
ROCK    	25.00%
ALL AVG 	26.87%
Gamma: 10
Weighted accuracy
POP     	21.52%
BLUES   	29.69%
METAL   	21.09%
HIPHOP  	19.49%
ROCK    	18.56%
ALL AVG 	22.07%
Gamma: 100
Weighted accuracy
POP     	13.59%
BLUES   	23.98%
METAL   	14.67%
HIPHOP  	19.49%
ROCK    	14.22%
ALL AVG 	17.19%
Gamma: 1000
Weighted accuracy
POP     	13.37%
BLUES   	20.10%
METAL   	12.07%
HIPHOP  	13.80%
ROCK    	12.44%
ALL AVG 	14.36%
CPU times: user 39min 28s, sys: 2min 33s, total: 42min 2s
Wall time: 5min 17s


In [54]:
%%time
key_detection_pipeline_pre(feature_dict, 
                           gammas = [1,10,100,1000],
                           key_template = KS_key_template,
                           match_method='dot',
                           blue_tone=True,
                           weight_eval = True,
                           out_file="Q5_blue_tone_remove_dot_weight.txt")

pop
blues
metal
hiphop
rock
Gamma: 1
Weighted accuracy
POP     	35.65%
BLUES   	19.18%
METAL   	33.37%
HIPHOP  	23.80%
ROCK    	26.67%
ALL AVG 	27.73%
Gamma: 10
Weighted accuracy
POP     	34.89%
BLUES   	20.20%
METAL   	38.04%
HIPHOP  	21.90%
ROCK    	28.89%
ALL AVG 	28.79%
Gamma: 100
Weighted accuracy
POP     	27.83%
BLUES   	20.10%
METAL   	22.17%
HIPHOP  	17.72%
ROCK    	23.33%
ALL AVG 	22.23%
Gamma: 1000
Weighted accuracy
POP     	24.13%
BLUES   	18.67%
METAL   	13.48%
HIPHOP  	16.33%
ROCK    	17.56%
ALL AVG 	18.03%
CPU times: user 39min 28s, sys: 2min 33s, total: 42min 1s
Wall time: 5min 16s
